In [1]:
import copy
import time

import numpy as np

from cv2 import cv2
from config.Config import Config
from mapping.mapper import Mapper
from agent.agent_handler import AgentHandler
from utils.util_functions import stateCoordsToName
from grid_world_generator.grid_world import GridWorld
from region_assignment.k_mean_clustring import KMeanClustring
from utils.graph import get_closest_vertex_coords_on_graph_from_pos
from region_assignment.hungarian_region_assignment import HungarianRegionAssignment
from occupancy_grid_generator.occupancy_grid_generator import OccupancyGridGenerator

# OCCUPANCY GRID GENERATOR

In [2]:
occupancy_grid = OccupancyGridGenerator()

occupancy_grid.generate_occupancy_grid()

# occupancy_grid.show_occupancy_grid_without_obs()
# occupancy_grid.show_occupancy_grid_with_obs()

temp_occupancy_grid_without_obs = occupancy_grid.get_occupancy_grid_without_obs()
temp_occupancy_grid_with_obs = occupancy_grid.get_occupancy_grid_with_obs()

# GRID WORLD

In [3]:
X_DIM = int(Config.GRID_WIDTH/Config.EDGE_COST)
Y_DIM = int(Config.GRID_LEN/Config.EDGE_COST)
VIEWING_RANGE = Config.SENSOR_RANGE
print("Edge Cost:", Config.EDGE_COST, ", Width:", X_DIM, ", Height:", \
      Y_DIM, ", Viewing Range:", VIEWING_RANGE)

graph_list = []

for i in range(Config.NO_OF_AGENTS):

    graph_list.append(GridWorld(X_DIM, Y_DIM, Config.EDGE_COST, temp_occupancy_grid_without_obs))

    graph_list[-1].run()
    #     temp_graph = copy.copy(graph.get_graph())
#     graph_list[-1].show_nodes_on_occupancy_grid()
# graph.show_nodes_and_edges_with_obs_on_occupancy_grid()
# graph.show_nodes_and_all_traversable_edges()
# temp_grid_with_nodes = graph_list[-1].get_occupancy_grid_with_nodes()
# temp_grid_with_nodes_and_edges_with_obs = graph_list[-1].get_occupancy_grid_with_nodes_and_edges_with_obs()
# temp_grid_with_nodes_and_all_traversable_edges = graph_list[-1].get_occupancy_grid_with_nodes_and_all_traversable_edges()
# print(graph.graph['x0y0'])
temp_graph_list = copy.copy(graph_list)

Edge Cost: 40 , Width: 25 , Height: 15 , Viewing Range: 40


# K-MEAN-CLUSTRING

In [4]:
regions = KMeanClustring(temp_occupancy_grid_without_obs)

regions.find_regions()

temp_regions_xy_points = regions.get_regions_xy_points()
temp_region_centroids = regions.get_centroids()
temp_color_map = regions.get_color_map()
temp_grid_with_regions = copy.copy(regions.get_grid_with_regions())

In [5]:
# regions.show_regions_with_centroids()
# regions.show_regions()

ind = 0
for el in temp_regions_xy_points:
    print("Region", ind, "indices:", el)
    ind += 1
print("Region centroids:\n", temp_region_centroids)
print("Region color map:\n", temp_color_map)

Region 0 indices: (array([103, 103, 103, ..., 509, 509, 509]), array([648, 649, 650, ..., 919, 920, 921]))
Region 1 indices: (array([103, 103, 103, ..., 509, 509, 509]), array([374, 375, 376, ..., 645, 646, 647]))
Region 2 indices: (array([103, 103, 103, ..., 509, 509, 509]), array([103, 104, 105, ..., 371, 372, 373]))
Region centroids:
 [[785 306]
 [510 306]
 [237 306]]
Region color map:
 [[152, 6, 145], [3, 83, 125], [251, 141, 128]]


# AGENT HANDLER

In [6]:
agenthandler = AgentHandler(graph_list)

# REGION ASSIGNMENT

In [7]:
cost_matrix = np.zeros(shape=[Config.NO_OF_AGENTS, Config.NO_OF_AGENTS])

print("Agent Positions:")

for i in range(Config.NO_OF_AGENTS):
    temp_agent_pos = agenthandler.get_pos_of_agent(i)
    a = np.array([temp_agent_pos['x'], temp_agent_pos['y']])
    print(a)
    for j in range(len(temp_region_centroids)):
        b = np.array(temp_region_centroids[j])
        cost_matrix[i,j] = np.linalg.norm(a-b)
        
cost_matrix

Agent Positions:
[480 560]
[80 40]
[240  40]


array([[396.91434844, 255.76551761, 351.51813609],
       [753.51244183, 505.6243665 , 308.87699817],
       [606.44950326, 379.0197884 , 266.01691676]])

In [8]:
hungaian_region_assignment = HungarianRegionAssignment(cost_matrix, temp_grid_with_regions)

hungaian_region_assignment.find_regions()
temp_regions_rows, temp_regions_cols = hungaian_region_assignment.get_regions()

goal_pos = [''] * Config.NO_OF_AGENTS

ind = 0
for el in temp_regions_cols:
    temp_x, temp_y = temp_region_centroids[el]
    
    temp_goal_pos = get_closest_vertex_coords_on_graph_from_pos(\
                        graph_list[ind].get_graph(), temp_x, temp_y, Config.EDGE_COST)
#     print(temp_goal_pos)
    goal_pos[ind] = stateCoordsToName(temp_goal_pos[1], temp_goal_pos[0], Config.EDGE_COST)
    
    ind += 1

print("Goal Position:\n", goal_pos, "\n")
print(temp_regions_rows, temp_regions_cols)
print(hungaian_region_assignment.get_total_cost())

Goal Position:
 ['x7y19', 'x7y5', 'x7y12'] 

[0 1 2] [0 2 1]
1084.8111350171066


In [15]:
hungaian_region_assignment.show_assigned_regions(agenthandler, temp_region_centroids)
# hungaian_region_assignment.show_assigned_regions(agenthandler, temp_region_centroids, temp_grid_with_nodes)

In [10]:
print("Region centroids:\n")
ind = 1
for el in temp_region_centroids:
    print("\tRegion:", ind, ", x:", el[0], ", y:", el[1])
    ind += 1
    
print("\nAgent Positions:\n")

for i in range(Config.NO_OF_AGENTS):
    temp_agent_pos = agenthandler.get_pos_of_agent(i)
    print("\tAgent:", i+1, ", x:", temp_agent_pos['x'], ", y:", temp_agent_pos['y'])
    
print("\nCost Matrix:\n\n", cost_matrix, "\n")

ind = 1
for el in temp_regions_cols:
    print("Region Assigned to Agent: {} is Region {}".format(ind, el))
    ind += 1
    
print("\nTotal cost:", hungaian_region_assignment.get_total_cost())

Region centroids:

	Region: 1 , x: 785 , y: 306
	Region: 2 , x: 510 , y: 306
	Region: 3 , x: 237 , y: 306

Agent Positions:

	Agent: 1 , x: 480 , y: 560
	Agent: 2 , x: 80 , y: 40
	Agent: 3 , x: 240 , y: 40

Cost Matrix:

 [[396.91434844 255.76551761 351.51813609]
 [753.51244183 505.6243665  308.87699817]
 [606.44950326 379.0197884  266.01691676]] 

Region Assigned to Agent: 1 is Region 0
Region Assigned to Agent: 2 is Region 2
Region Assigned to Agent: 3 is Region 1

Total cost: 1084.8111350171066


# D Star Lite

In [11]:
import heapq
from utils.util_functions import stateNameToCoords


def topKey(queue):
    queue.sort()
    # print(queue)
    if len(queue) > 0:
        return queue[0][:2]
    else:
        # print('empty queue!')
        return (float('inf'), float('inf'))


def heuristic_from_s(graph, id, s):
    x_distance = abs(int(id.split('x')[1][0]) - int(s.split('x')[1][0]))
    y_distance = abs(int(id.split('y')[1][0]) - int(s.split('y')[1][0]))
    return max(x_distance, y_distance)


def calculateKey(graph, id, s_current, k_m):
    return (min(graph.graph[id].g, graph.graph[id].rhs) + heuristic_from_s(graph, id, s_current) + k_m, min(graph.graph[id].g, graph.graph[id].rhs))


def updateVertex(graph, queue, id, s_current, k_m):
    s_goal = graph.goal
    if id != s_goal:
        min_rhs = float('inf')
        for i in graph.graph[id].children:
            min_rhs = min(
                min_rhs, graph.graph[i].g + graph.graph[id].children[i])
        graph.graph[id].rhs = min_rhs
    id_in_queue = [item for item in queue if id in item]
    if id_in_queue != []:
        if len(id_in_queue) != 1:
            raise ValueError('more than one ' + id + ' in the queue!')
        queue.remove(id_in_queue[0])
    if graph.graph[id].rhs != graph.graph[id].g:
        heapq.heappush(queue, calculateKey(graph, id, s_current, k_m) + (id,))


def computeShortestPath(graph, queue, s_start, k_m):
    while (graph.graph[s_start].rhs != graph.graph[s_start].g) or (topKey(queue) < calculateKey(graph, s_start, s_start, k_m)):
        # print(graph.graph[s_start])
        # print('topKey')
        # print(topKey(queue))
        # print('calculateKey')
        # print(calculateKey(graph, s_start, 0))
        k_old = topKey(queue)
        u = heapq.heappop(queue)[2]
        if k_old < calculateKey(graph, u, s_start, k_m):
            heapq.heappush(queue, calculateKey(graph, u, s_start, k_m) + (u,))
        elif graph.graph[u].g > graph.graph[u].rhs:
            graph.graph[u].g = graph.graph[u].rhs
            for i in graph.graph[u].parents:
                updateVertex(graph, queue, i, s_start, k_m)
        else:
            graph.graph[u].g = float('inf')
            updateVertex(graph, queue, u, s_start, k_m)
            for i in graph.graph[u].parents:
                updateVertex(graph, queue, i, s_start, k_m)
        # graph.printGValues()


def nextInShortestPath(graph, s_current):
    min_rhs = float('inf')
    s_next = None
    if graph.graph[s_current].rhs == float('inf'):
        print('You are done stuck')
    else:
        for i in graph.graph[s_current].children:
            # print(i)
            child_cost = graph.graph[i].g + graph.graph[s_current].children[i]
            # print(child_cost)
            if (child_cost) < min_rhs:
                min_rhs = child_cost
                s_next = i
        if s_next:
            return s_next
        else:
            raise ValueError('could not find child for transition!')


def scanForObstacles(graph, queue, s_current, scan_range, k_m):
    states_to_update = {}
    range_checked = 0
    if scan_range >= 1:
        for neighbor in graph.graph[s_current].children:
            neighbor_coords = stateNameToCoords(neighbor, Config.EDGE_COST)
            states_to_update[neighbor] = graph._cells[neighbor_coords[1]
                                                     ][neighbor_coords[0]]
        range_checked = 1
    # print(states_to_update)

    while range_checked < scan_range:
        new_set = {}
        for state in states_to_update:
            new_set[state] = states_to_update[state]
            for neighbor in graph.graph[state].children:
                if neighbor not in new_set:
                    neighbor_coords = stateNameToCoords(neighbor, Config.EDGE_COST)
                    new_set[neighbor] = graph._cells[neighbor_coords[1]
                                                    ][neighbor_coords[0]]
        range_checked += 1
        states_to_update = new_set

    new_obstacle = False
    for state in states_to_update:
        if states_to_update[state] < 0:  # found cell with obstacle
            # print('found obstacle in ', state)
            for neighbor in graph.graph[state].children:
                # first time to observe this obstacle where one wasn't before
                if(graph.graph[state].children[neighbor] != float('inf')):
                    neighbor_coords = stateNameToCoords(state, Config.EDGE_COST)
                    graph._cells[neighbor_coords[1]][neighbor_coords[0]] = -2
                    graph.graph[neighbor].children[state] = float('inf')
                    graph.graph[state].children[neighbor] = float('inf')
                    updateVertex(graph, queue, state, s_current, k_m)
                    new_obstacle = True
        # elif states_to_update[state] == 0: #cell without obstacle
            # for neighbor in graph.graph[state].children:
                # if(graph.graph[state].children[neighbor] != float('inf')):

    # print(graph)
    return new_obstacle


def moveAndRescan(graph, queue, s_current, scan_range, k_m):
    if(s_current == graph.goal):
        return 'goal', k_m
    else:
        s_last = s_current
        s_new = nextInShortestPath(graph, s_current)
        new_coords = stateNameToCoords(s_new, Config.EDGE_COST)

#         if(graph._cells[new_coords[1]][new_coords[0]] == -1):  # just ran into new obstacle
#             s_new = s_current  # need to hold tight and scan/replan first

#         results = scanForObstacles(graph, queue, s_new, scan_range, k_m)
        # print(graph)
        k_m += heuristic_from_s(graph, s_last, s_new)
        computeShortestPath(graph, queue, s_current, k_m)

        return s_new, k_m


def initDStarLite(graph, queue, s_start, s_goal, k_m):
    graph.graph[s_goal].rhs = 0
    heapq.heappush(queue, calculateKey(
        graph, s_goal, s_start, k_m) + (s_goal,))
    computeShortestPath(graph, queue, s_start, k_m)

    return (graph, queue, k_m)


In [12]:
grid_mapper = Mapper(global_grid=temp_occupancy_grid_without_obs)

display = copy.copy(temp_occupancy_grid_without_obs)
font = cv2.FONT_HERSHEY_SIMPLEX

s_start_names = [''] * Config.NO_OF_AGENTS
s_current_names = [''] * Config.NO_OF_AGENTS
s_goal_names = copy.copy(goal_pos)
k_m_list = [0] * Config.NO_OF_AGENTS
s_new_names = [''] * Config.NO_OF_AGENTS
s_last_names = s_start_names
queue_list = [[]] * Config.NO_OF_AGENTS
goal_reached_status_list = [False] * Config.NO_OF_AGENTS
graph_list = copy.copy(temp_graph_list)
temp_coord = [0,0]

for i in range(Config.NO_OF_AGENTS):
    
    temp_coord = agenthandler.get_pos_of_agent(i)
    s_start_names[i] = stateCoordsToName(temp_coord['y'], temp_coord['x'], Config.EDGE_COST)

print("START NODE NAMES:")
print(s_start_names, "\n")
print("GOAL NODE NAMES:")
print(s_goal_names, "\n")
print("Number of Graphs in list:", len(graph_list), "\n")

for i in range(len(graph_list)):
    graph_list[i].setStart(s_start_names[i])
    graph_list[i].setGoal(s_goal_names[i])
    graph_list[i], queue_list[i], k_m_list[i] = initDStarLite(graph_list[i], queue_list[i], \
                                                  s_start_names[i], s_goal_names[i], k_m_list[i])

s_current_names = s_start_names
for i in range(len(s_current_names)):
    pos_coords = stateNameToCoords(s_current_names[i], Config.EDGE_COST)

    print(s_current_names[i], pos_coords)

print("")
while s_new_names[0] != s_goal_names[0] or s_new_names[1] != s_goal_names[1] or s_new_names[2] != s_goal_names[2]:
    
    display = copy.copy(temp_occupancy_grid_without_obs)
    
    for i in range(Config.NO_OF_AGENTS):

        temp_pos = agenthandler.get_pos_of_agent(i)

        temp_pos_x = temp_pos['x']
        temp_pos_y = temp_pos['y']

        if i == 0:
            color_b,color_g,color_r = 255,0,0
        elif i == 1:
            color_b,color_g,color_r = 0,255,0
        else:
            color_b,color_g,color_r = 0,0,255

        img = cv2.ellipse(display,(temp_pos_x,temp_pos_y),(10,10),0,15,345,(color_b,color_g,color_r),-1)
        
        grid_mapper.map_grid(agent_no=i, agent_pos=temp_pos)

    temp_img = cv2.resize(img, (512, 512))

    mapped_grid = grid_mapper.get_mapped_grid()

    temp_mapped_img = cv2.resize(mapped_grid, (512, 512))

    numpy_horizontal = np.hstack((temp_img, temp_mapped_img))

    cv2.imshow('image', numpy_horizontal)
    k = cv2.waitKey(1) & 0xFF
    
    if k == ord('q'):
        break

    time.sleep(0.5)
    
    for i in range(len(graph_list)):
        
        if goal_reached_status_list[i] is False:
            s_new_names[i], k_m_list[i] = moveAndRescan(graph_list[i], queue_list[i], s_current_names[i], \
                                                        VIEWING_RANGE, k_m_list[i])
                
        s_current_names[i] = s_new_names[i]
        
#         if i == 0 and goal_reached_status_list[i] == False:
#             print(i, s_new_names[i], k_m_list[i])
#         if i == 1 and goal_reached_status_list[i] == False:
#             print("\t\t", i, s_new_names[i], k_m_list[i])
#         if i == 2 and goal_reached_status_list[i] == False:
#             print("\t\t\t\t", i, s_new_names[i], k_m_list[i])
        
        temp_coord = stateNameToCoords(s_current_names[i], Config.EDGE_COST)
        agenthandler.set_pos_of_agent(i, temp_coord[1], temp_coord[0])
            
        if s_current_names[i] == s_goal_names[i]:
                goal_reached_status_list[i] = True

while(1):
    
    if k == ord('q'):
        break
    else:
        cv2.imshow('image', numpy_horizontal)
        k = cv2.waitKey(1) & 0xFF
        time.sleep(1)

print("Shutting Down...")
cv2.destroyAllWindows()
print("Shutting Down Successfull.")

START NODE NAMES:
['x13y11', 'x0y1', 'x0y5'] 

GOAL NODE NAMES:
['x7y19', 'x7y5', 'x7y12'] 

Number of Graphs in list: 3 

x13y11 [560, 480]
x0y1 [40, 80]
x0y5 [40, 240]

Shutting Down...
Shutting Down Successfull.


# GRID MAPPER

In [13]:
# grid_mapper = Mapper(global_grid=temp_occupancy_grid_with_obs)

# display = copy.copy(temp_occupancy_grid_with_obs)
# font = cv2.FONT_HERSHEY_SIMPLEX

# while(1):

#     display = copy.copy(temp_occupancy_grid_with_obs)

#     for i in range(Config.NO_OF_AGENTS):

#         temp_pos = agenthandler.get_pos_of_agent(i)

#         temp_pos_x = temp_pos['x']
#         temp_pos_y = temp_pos['y']

#         if i == 0:
#             color_b,color_g,color_r = 255,0,0
#         elif i == 1:
#             color_b,color_g,color_r = 0,255,0
#         else:
#             color_b,color_g,color_r = 0,0,255

#         cv2.circle(temp_occupancy_grid_with_obs, (temp_pos_x, temp_pos_y), 1, (color_b,color_g,color_r), -1)

#         img = cv2.ellipse(display,(temp_pos_x,temp_pos_y),(10,10),0,15,345,(color_b,color_g,color_r),-1)

#         grid_mapper.map_grid(agent_no=i, agent_pos=temp_pos)
    
#     temp_img = cv2.resize(img, (512, 512))

#     mapped_grid = grid_mapper.get_mapped_grid()
    
#     temp_mapped_img = cv2.resize(mapped_grid, (512, 512))

#     numpy_horizontal = np.hstack((temp_img, temp_mapped_img))

#     cv2.imshow('image', numpy_horizontal)
#     k = cv2.waitKey(1) & 0xFF

#     if k == ord('q'):
#         break
#     if k == ord('1'):
#         agent_number = 0
#     if k == ord('2'):
#         agent_number = 1
#     if k == ord('3'):
#         agent_number = 2
#     if k == ord('w'):
#         agenthandler.move_agent(agent_number, 0, -5)
#     if k == ord('s'):
#         agenthandler.move_agent(agent_number, 0, 5)
#     if k == ord('a'):
#         agenthandler.move_agent(agent_number, -5, 0)
#     if k == ord('d'):
#         agenthandler.move_agent(agent_number, 5, 0)


# cv2.destroyAllWindows()
# # grid_mapper.show_mapped_grid()

# # cv2.imshow('Occupancy_grid',img)
# # cv2.waitKey(0)
# # cv2.destroyAllWindows()

In [14]:
# black_img = np.zeros(shape=[512,512,3], dtype=np.uint8)
# font = cv2.FONT_HERSHEY_SIMPLEX
# img = cv2.putText(black_img,'OpenCV',(10,500), font, 4,(255,255,255),2,cv2.LINE_AA)

# cv2.imshow('Ellipse', img)

# cv2.waitKey(0)
# cv2.destroyAllWindows()